In [1]:
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
import os
import cv2

In [2]:
prototxtPath=os.path.sep.join([r'C:\Users\KIIT\Videos\Face_Mask_Detection\face_detector','deploy.prototxt'])
weightsPath=os.path.sep.join([r'C:\Users\KIIT\Videos\Face_Mask_Detection\face_detector','res10_300x300_ssd_iter_140000.caffemodel'])


In [3]:
imagedetector=cv2.dnn.readNet(prototxtPath,weightsPath)

In [11]:
model=load_model(r'C:\Users\KIIT\Videos\Face_Mask_Detection\mask_detector.model')

In [12]:
image=cv2.imread(r'C:\Users\KIIT\Videos\Face_Mask_Detection\example\public.jpg')
image

array([[[123, 170, 168],
        [ 81, 123, 122],
        [161, 191, 192],
        ...,
        [210, 211, 207],
        [211, 208, 204],
        [208, 205, 201]],

       [[123, 170, 168],
        [ 81, 123, 122],
        [161, 191, 192],
        ...,
        [212, 211, 207],
        [211, 208, 204],
        [209, 206, 202]],

       [[123, 170, 168],
        [ 81, 123, 122],
        [161, 191, 192],
        ...,
        [215, 212, 207],
        [212, 209, 204],
        [211, 207, 202]],

       ...,

       [[ 21,  14,  57],
        [ 20,  16,  52],
        [ 45,  42,  68],
        ...,
        [ 35,  20,  11],
        [ 43,  27,  20],
        [ 36,  20,  14]],

       [[ 43,  36,  79],
        [ 14,  10,  46],
        [ 48,  45,  71],
        ...,
        [ 35,  20,  11],
        [ 43,  27,  20],
        [ 36,  20,  14]],

       [[ 94,  87, 130],
        [ 42,  38,  74],
        [ 39,  36,  62],
        ...,
        [ 35,  20,  11],
        [ 43,  27,  20],
        [ 36,  20,  14]]

In [13]:
(h,w)=image.shape[:2]
(h,w)

(180, 280)

In [14]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))
blob

array([[[[  19.,  -19.,   44., ...,  106.,  107.,  104.],
         [  19.,  -19.,   44., ...,  107.,  107.,  104.],
         [  19.,  -19.,   44., ...,  108.,  107.,  105.],
         ...,
         [ -61.,  -87.,  -62., ...,  -68.,  -62.,  -68.],
         [ -31.,  -69.,  -64., ...,  -68.,  -62.,  -68.],
         [ -10.,  -57.,  -65., ...,  -68.,  -62.,  -68.]],

        [[  -7.,  -49.,    2., ...,   33.,   31.,   28.],
         [  -7.,  -49.,    3., ...,   33.,   31.,   28.],
         [  -7.,  -49.,    3., ...,   34.,   31.,   29.],
         ...,
         [-141., -164., -138., ..., -156., -151., -157.],
         [-111., -146., -140., ..., -156., -151., -157.],
         [ -90., -134., -141., ..., -156., -151., -157.]],

        [[  45.,    4.,   57., ...,   83.,   81.,   78.],
         [  45.,    3.,   57., ...,   83.,   80.,   78.],
         [  45.,    4.,   57., ...,   84.,   81.,   79.],
         ...,
         [ -44.,  -74.,  -56., ..., -111., -104., -109.],
         [ -13.,  -56.,  -

In [15]:
blob.shape

(1, 3, 300, 300)

In [16]:
imagedetector.setInput(blob)
detections=imagedetector.forward()
detections

array([[[[0.        , 1.        , 0.9981135 , ..., 0.58194125,
          0.427413  , 0.88312006],
         [0.        , 1.        , 0.9930099 , ..., 0.35360545,
          0.8363572 , 0.6033377 ],
         [0.        , 1.        , 0.9928404 , ..., 0.1538603 ,
          0.26527372, 0.31458342],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [17]:
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX),min(h-1,endY))
        
        face=image[startY:endY , startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else(0,0,255)
        
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()